PHASE 2

In [ ]:
!pip install -q -U groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq
from google.colab import files

GROQ_API_KEY = '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'

client = Groq(
    api_key=GROQ_API_KEY,
)

# Function to upload a file
def upload_file():
    print("Please upload the 'asl_input.txt' file.")
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the filename

# Function to read ASL words from a file
def read_asl_words(file_path):
    with open(file_path, "r") as file:
        content = file.read().strip()
    asl_words = content.replace(",", " ").split()  # Handle commas or spaces
    return asl_words

# Function to correct grammar and structure sentences
def correct_grammar(asl_words):
    prompt = (
        "The following list of words were recognized from American Sign Language (ASL). "
        "Correct the grammar and organize them into separate sentences with proper grammar. "
        "Do not use any other punctuation other than full stops. "
        "Here are the ASL words: "
        f"{', '.join(asl_words)}"
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="llama-3.3-70b-versatile",
    )

    return chat_completion.choices[0].message.content

# Function to convert corrected English sentences into ISL format
def convert_to_isl(corrected_sentences):
    prompt = (
        "Convert the following English sentences into Indian Sign Language (ISL) format following Subject-Object-Verb (SOV) structure. "
        "Output only the final ISL words and nothing else. "
        "Separate the sentences using full stops. "
        f"{corrected_sentences}"
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "user", "content": prompt}
        ],
        model="llama-3.3-70b-versatile",
    )

    return chat_completion.choices[0].message.content

# Function to format ISL output
def format_isl_output(input_file, output_file):
    with open(input_file, "r") as file:
        text = file.read()

    formatted_text = text.replace(". ", ".")  # Trim spaces after full stops
    formatted_text = formatted_text.replace(" ", ",")  # Replace remaining spaces with commas
    formatted_text = formatted_text.replace(".", ";")  # Replace full stops with semicolons

    with open(output_file, "w") as file:
        file.write(formatted_text)

# Upload file and get file path
uploaded_file = upload_file()
asl_input_path = f"/content/{uploaded_file}"

# File paths
input_file_path = "/content/input.txt"  # Intermediate ISL output file
output_file_path = "/content/input_words.txt"  # Final formatted ISL output file

# Step 1: Read ASL words from file
asl_words = read_asl_words(asl_input_path)

# Step 2: Correct grammar in ASL words
corrected_sentences = correct_grammar(asl_words)

# Step 3: Convert corrected sentences to ISL format
isl_output = convert_to_isl(corrected_sentences)

# Save ISL output to a text file
with open(input_file_path, "w") as file:
    file.write(isl_output)

# Step 4: Format ISL output and save
format_isl_output(input_file_path, output_file_path)

print(f"Formatted output saved to {output_file_path}")

Please upload the 'asl_input.txt' file.


Saving gloss.txt to gloss.txt
Formatted output saved to /content/input_words.txt


PHASE 3

In [ ]:
!pip install pytube
!pip install yt_dlp moviepy ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.6 MB/s eta 0:00:00


In [ ]:
import os
import json
import yt_dlp
import moviepy.editor as mp
import warnings
from google.colab import drive
from google.colab import files
import re

  if event.key is 'enter':



In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === STEP 1: DEFINE PATHS === #
LETTER_IMAGES_DIR = "/content/drive/MyDrive/isl_fingerspell"  # Folder with letter images
PLAYLIST_JSON_PATH = "/content/drive/MyDrive/final_modified_playlist_videos.json"  # JSON with ISL words and videos
INPUT_WORDS_FILE = "/content/input_words.txt"  # File containing words & sentences
SAVE_DIR = "/content/"  # Directory to save individual word videos

# Ensure save directory exists
os.makedirs(SAVE_DIR, exist_ok=True)

# Check if playlist JSON exists
if not os.path.exists(PLAYLIST_JSON_PATH):
    raise FileNotFoundError(f"❌ Playlist file '{PLAYLIST_JSON_PATH}' not found! Upload it to Colab.")

# Check if input words file exists
if not os.path.exists(INPUT_WORDS_FILE):
    raise FileNotFoundError(f"❌ Input words file '{INPUT_WORDS_FILE}' not found! Create it first.")

# Load playlist JSON
with open(PLAYLIST_JSON_PATH, "r") as file:
    playlist_data = json.load(file)

# Extract dictionary of words to video URLs
word_to_video = {key.strip().lower(): value for key, value in playlist_data.items()}

# Read input file and process sentences
with open(INPUT_WORDS_FILE, "r", encoding="utf-8") as file:
    raw_content = file.read().strip().lower()

# Split sentences using `;`
sentences = [sentence.strip() for sentence in raw_content.split(";") if sentence.strip()]

if not sentences:
    raise ValueError("❌ No sentences found in input_words.txt. Please add text and try again.")

print(f"✅ Loaded {len(sentences)} sentences from '{INPUT_WORDS_FILE}'.")

# Create temporary directory for video downloads
TEMP_DIR = "/tmp/isl_videos"
os.makedirs(TEMP_DIR, exist_ok=True)

# === FUNCTION TO DOWNLOAD YOUTUBE VIDEO === #
def download_video(word, url):
    """Downloads a YouTube video for a given word and saves it separately."""
    temp_path = os.path.join(TEMP_DIR, f"{word}.mp4")  # Temporary path
    save_path = os.path.join(SAVE_DIR, f"{word}.mp4")  # Final location

    ydl_opts = {
        "outtmpl": temp_path,
        "format": "bestvideo+bestaudio/best",
        "merge_output_format": "mp4",
        "quiet": True
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Move to save directory if download is successful
    if os.path.exists(temp_path):
        os.rename(temp_path, save_path)
        return save_path
    else:
        return None

# === FUNCTION TO FETCH REQUESTED LETTER IMAGES === #
def get_requested_images(folder_path, requested_letters):
    """Fetches specific images for the given letters. Returns a list of valid image file paths."""
    images = []
    for letter in requested_letters:
        img_path = os.path.join(folder_path, f"{letter}.jpg")  # Assuming filenames are "a.jpg", "b.jpg", etc.
        if os.path.exists(img_path):
            images.append(img_path)
        else:
            print(f"⚠️ Warning: No image found for letter '{letter}'")

    return images if images else None

# === FUNCTION TO CREATE A WORD VIDEO USING LETTER IMAGES === #
def create_word_video(word):
    """Creates a video clip from letter images if no video is found in the playlist."""
    try:
        letter_images = get_requested_images(LETTER_IMAGES_DIR, list(word))
        if not letter_images:
            return None

        letter_clips = [
            mp.CompositeVideoClip([mp.ColorClip((1280, 720), color=(0, 0, 0), duration=1),
                                   mp.ImageClip(img).resize(height=500).set_position("center").set_duration(1)])
            for img in letter_images
        ]

        final_clip = mp.concatenate_videoclips(letter_clips, method="compose")
        output_path = os.path.join(SAVE_DIR, f"{word}.mp4")
        final_clip.write_videofile(output_path, fps=24, codec="libx264")

        return output_path
    except Exception as e:
        print(f"⚠️ Error processing word '{word}': {e}")
        return None

# === PROCESS SENTENCES === #
sentence_clips = []

for sentence in sentences:
    words = [word.strip() for word in sentence.split(",") if word.strip()]
    word_clips = []

    for word in words:
        video_path = os.path.join(SAVE_DIR, f"{word}.mp4")

        if not os.path.exists(video_path):  # If video isn't already saved
            if word in word_to_video:
                print(f"✅ Found ISL video for '{word}', downloading...")
                video_path = download_video(word, word_to_video[word])
            else:
                print(f"⚠️ '{word}' not found in playlist. Using letter images.")
                video_path = create_word_video(word)

        if video_path and os.path.exists(video_path):
            clip = mp.VideoFileClip(video_path).resize(height=720)  # Standardize size
            word_clips.append(clip)

    if word_clips:
        sentence_clip = mp.concatenate_videoclips(word_clips, method="compose")
        sentence_clips.append(sentence_clip)

# === CONCATENATE SENTENCES INTO FINAL VIDEO === #
if sentence_clips:
    final_video = mp.concatenate_videoclips(sentence_clips, method="compose")
    final_output_path = "/content/final_output.mp4"
    final_video.write_videofile(final_output_path, fps=24, codec="libx264")

    print(f"🎉 Final video saved at: {final_output_path}")
    files.download(final_output_path)

else:
    print("❌ No valid videos found. Exiting.")


✅ Loaded 1 sentences from '/content/input_words.txt'.
✅ Found ISL video for 'i', downloading...
✅ Found ISL video for 'good', downloading...
✅ Found ISL video for 'day', downloading...
✅ Found ISL video for 'have', downloading...
Moviepy - Building video /content/final_output.mp4.
MoviePy - Writing audio in final_outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/final_output.mp4



t:  32%|███▏      | 116/367 [00:12<00:22, 11.15it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/i.mp4, 6220800 bytes wanted but 0 bytes read,at frame 120/122, at time 4.80/4.86 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/final_output.mp4
🎉 Final video saved at: /content/final_output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PHASE 3 PREPROCESSING -> playlist_videos dictionary (only had to be run once, output now stored in google drive and linked to step 3)

In [ ]:
#STEP 3.0.1 getting playlist_videos.json

In [ ]:
# Install yt-dlp if not installed
#!pip install -q yt-dlp

import json
import yt_dlp

# YouTube Playlist URL
playlist_url = "https://youtube.com/playlist?list=PLFjydPMg4Dapq9vcdmGyHs8uJhiqMgUrX&si=uurYj7GMgtnqlSQm"

# Define file paths within Colab's /content directory
playlist_json = "/content/playlist_videos.json"
titles_json = "/content/video_titles.json"

# Function to fetch all video details from the playlist and store them
def fetch_playlist_videos(playlist_url):
    print("Fetching playlist videos, this may take some time...")

    ydl_opts = {
        "quiet": True,
        "extract_flat": True,
        "force_generic_extractor": True
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        playlist_data = ydl.extract_info(playlist_url, download=False)

    # Store videos as list of dictionaries
    video_data = [{"title": video["title"], "url": video["url"]} for video in playlist_data["entries"]]
    video_titles = [video["title"] for video in playlist_data["entries"]]

    # Save to JSON files inside Colab's session storage
    with open(playlist_json, "w") as f:
        json.dump(video_data, f, indent=4)

    with open(titles_json, "w") as f:
        json.dump(video_titles, f, indent=4)

    print(f"✅ Playlist data saved in '{playlist_json}' and '{titles_json}' (in Colab's session storage)")
    return video_data, video_titles

# Fetch playlist data and save in /content
video_data, video_titles = fetch_playlist_videos(playlist_url)

Fetching playlist videos, this may take some time...
✅ Playlist data saved in '/content/playlist_videos.json' and '/content/video_titles.json' (in Colab's session storage)


In [ ]:
#STEP 3.0.2 MODIFY and format playlist_videos.json and output modified_playlist_videos.json

In [ ]:
import re
import json

# Path of the saved playlist file
filename = "/content/drive/MyDrive/playlist_videos.json"

# Read the file
with open(filename, "r", encoding="utf-8") as file:
    content = file.read()

# Define a list of phrases to remove before processing parentheses
phrases_to_remove_first = ["sign 1", "sign 2","sign 3","sign 4","sign 5","sign 6"]

# Function to clean up text
def process_match(match):
    phrase = match.group(1)

    # Remove "sign 1" and "sign 2" before handling parentheses
    for unwanted in phrases_to_remove_first:
        phrase = re.sub(rf'\b{re.escape(unwanted)}\b', '', phrase, flags=re.IGNORECASE)

    # Remove everything inside parentheses along with the parentheses
    phrase = re.sub(r'\s*\(.*?\)\s*', '', phrase)

    # Remove extra spaces created due to removals
    phrase = re.sub(r'\s+', ' ', phrase).strip()

    # Replace spaces with underscores
    phrase = phrase.replace(" ", "_")

    return f'"{phrase}"'

# Apply transformations
modified_content = re.sub(r'"(.*?)"', process_match, content)

# Save the modified content in Colab's session storage
output_filename = "/content/modified_playlist_videos.json"
with open(output_filename, "w", encoding="utf-8") as file:
    file.write(modified_content)

print(f"✅ Modified JSON file saved at '{output_filename}' in Colab's session storage.")

✅ Modified JSON file saved at '/content/modified_playlist_videos.json' in Colab's session storage.
